In [1]:
import cv2 as cv
import numpy as np
import pickle
from matplotlib import pyplot as plt

with open('trimmed_inputs.pkl', 'rb') as fp:
    flar_scans = pickle.load(fp)
    print(flar_scans.shape)
with open('trimmed_outputs.pkl', 'rb') as fp:
    seg_scans = pickle.load(fp)
    print(seg_scans.shape)

scores = []
for idx in range(25650):
    # check to make sure there is tumor present
    if np.sum(seg_scans[idx][0]) == 0:
        continue
    img = flar_scans[idx][0]
    # preprocess to make sure all values are between 0 and 255 uint8 values
    process_img = np.uint8(img)     
    
    # add a blur to smooth out noise
    blur = cv.bilateralFilter(process_img,9,75,75)
    # if it's above the threshold get rid of it, because it's probably actual
    # brain and not a tumor, otherwise keep everything the same
    ret, thresh5 = cv.threshold(process_img, 0, 255, cv.THRESH_TOZERO_INV+cv.THRESH_OTSU)
    
    # Here, we convert any cell that wasn't cleared as a tumor (1)
    # and all else are set to not a tumor (0) to handle possible floats
    for i in range(32):
        for j in range(32):
            if thresh5[i][j] > 0:
                thresh5[i][j] = 1
            else:
                thresh5[i][j] = 0
    # Here we normalize our true tumor segment to one values
    # because we only care about tumor or not tumor & so that our IOU metric works properly
    cpy = np.uint8(np.copy(seg_scans[idx][0]))
    for i in range(32):
        for j in range(32):
            if seg_scans[idx][0][i][j] > 0:
                cpy[i][j] = 1
            else:
                cpy[i][j] = 0
    # Logical IOU metric implementation
    intersection = np.logical_and(cpy, thresh5)
    union = np.logical_or(cpy, thresh5)
    numerator = np.sum(intersection)
    denominator = np.sum(union)
    iou_score = np.sum(intersection) / np.sum(union)
    scores.append(iou_score)
#print out average IOU score
print(sum(scores)/len(scores))

(25650, 4, 32, 32)
(25650, 1, 32, 32)


<Figure size 640x480 with 1 Axes>

0.24283325430015096


done
